In [ ]:
import param
import numpy as np
import datashader as ds
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
from colorcet import cm_n

from earthsim.analysis import LineCrossSection, SurfaceCrossSection
from earthsim.io import read_3dm_mesh, read_mesh2d

hv.extension('bokeh')

%output holomap='scrubber'
%opts Image RGB [width=500 height=400 colorbar=True] Curve {+framewise} [height=400 width=400] VLine (color='black')
%opts Path (line_width=3 color='black') NdOverlay [legend_limit=0] Image {+framewise} [height=400 width=500]

This notebook provides a demo of ``LineCrossSection`` and ``SurfaceCrossSection`` helpers which allows taking cross-sections of a mesh along linestrings drawn using the ``PolyDraw`` tool. The helper rasterize the data at a fixed resolution and then samples the data at the same resolution given the underlying line strings. The ``resolution`` may be defined in the units of the underlying
coordinate system.

## Example 1: A static TriMesh

As a simple example we will pass the static TriMesh to the ``CrossSector`` and display it. If now draw poly lines on the plot (left) we can see the depth along the path appear on the right.

In [ ]:
path = '../data/Chesapeake_and_Delaware_Bays.3dm'
tris, verts = read_3dm_mesh(path)
points = gv.operation.project_points(gv.Points(verts, vdims=['z']))
trimesh = hv.TriMesh((tris, points))

sector = LineCrossSection(trimesh)
sector.view()

## Example 2: A time-varying mesh

The ``CrossSector`` also allows working with time-varying meshes. Here we will modify the static TriMesh data with a time-varying random offset to demonstrate that the ``CrossSector`` works even for data that is evolving temporally.

In [ ]:
fpath = '../data/SanDiego_Mesh/SanDiego.3dm'
tris, verts = read_3dm_mesh(fpath, skiprows=2)
points = gv.operation.project_points(gv.Points(verts, vdims=['z'], crs=ccrs.UTM(11)))
trimesh = gv.TriMesh((tris, points))

fpath = '../data/SanDiego_Mesh/SanDiego_ovl.dat'
dfs = read_mesh2d(fpath)

points = gv.operation.project_points(gv.Points((verts.x, verts.y), crs=ccrs.UTM(11)))

def time_mesh(time):
    depth_points = points.add_dimension('Velocity', 0, dfs[time].values[:, 0], vdim=True)
    return gv.TriMesh((tris, depth_points), crs=ccrs.GOOGLE_MERCATOR)

meshes = hv.DynamicMap(time_mesh, kdims='Time').redim.values(Time=sorted(dfs.keys()))

In [ ]:
sector = LineCrossSection(meshes, resolution=100)
sector.view(cmap=cm_n.rainbow, shade=True)

## Example 3: Sampling a Surface

Instead of sampling individual Curve elements for each time, we can also take cross-sections across time returning an Image plotting the sampled value across both distance and time. Here we will subclass the ``CrossSector`` and override the ``sample`` method to apply the sampling for all time values and return an ``Image``.

In [ ]:
sector = SurfaceCrossSection(meshes, resolution=100)
sector.view(cmap=cm_n.rainbow, shade=False)